# 一. 验证命名实体识别（NER）模型输出的准确性和可靠性的方法总结

1. 交叉验证

数据划分: 将数据集划分为训练集、验证集和测试集。确保测试集是未见过的数据，用于评估模型的泛化能力。

交叉验证: 使用K折交叉验证，在多个数据子集上训练和评估模型，以确保模型在不同数据上的一致性表现。

2. 使用标注数据集

人工标注: 创建一个包含手动标注实体的数据集，并将其用作参考。
可以使用多名标注者对同一数据进行标注，然后通过一致性检查来确保标注的准确性。

评估指标: 使用常见的评估指标来评估模型输出的准确性，如：

精确率 (Precision): 正确识别的实体数量 / 被识别的实体总数量

召回率 (Recall): 正确识别的实体数量 / 实际实体总数量

F1分数: 精确率和召回率的调和平均，F1 = 2 * (精确率 * 召回率) / (精确率 + 召回率)

3. 混淆矩阵

混淆矩阵: 构建混淆矩阵来可视化模型的预测结果，包括真实标签与预测标签之间的关系。通过混淆矩阵，可以直观地查看哪些实体类型的识别效果较差，并分析具体问题。

4. 统计分析

错误分析: 统计模型在测试集上识别出的错误，分析常见的错误类型（如假阳性和假阴性），并寻找改进模型的方向。

记录模型的预测和真实标签，评估输出的一致性。

5. 人工审查

人工审查: 随机抽取一定比例的预测结果进行人工审查，检查提取的实体是否符合预期。 通过专家审核来提高模型输出的可信度，尤其是在对某些特定行业的实体提取上。
6. 置信度评分

置信度机制: 在NER模型中，通常可以为每个实体的预测输出一个置信度评分。根据置信度阈值筛选输出结果，确保只保留高置信度的预测。


##### 1. 检查提取的数值是否符合标准：

In [ ]:
def validate_extracted_data(extracted_data, standards):
    """
    验证提取的数据是否符合已知标准

    Args:
        extracted_data (dict): 提取的ESG信息
        standards (dict): 已知的标准信息

    Returns:
        list: 不符合标准的数据项
    """
    violations = []

    # 检查碳排放量
    if 'carbon_emission' in extracted_data:
        emission_value = extracted_data['carbon_emission']
        if emission_value < standards['carbon_emission']['min'] or emission_value > standards['carbon_emission']['max']:
            violations.append(f"碳排放量 {emission_value} 不符合标准 ({standards['carbon_emission']['min']} - {standards['carbon_emission']['max']})")
    
    return violations

# 示例提取数据
extracted_data = {
    "carbon_emission": 1500
}

# 示例标准
standards = {
    "carbon_emission": {
        "min": 1000,
        "max": 2000
    }
}

# 验证提取的数据
violations = validate_extracted_data(extracted_data, standards)
print("验证结果:", violations)

##### 2. 置信评分机制
目标: 为提取的数据点分配一个置信度评分，以便于判断数据的可靠性。

实现步骤：

设计评分模型: 可以通过训练模型来预测提取结果的置信度。例如，使用分类模型来评估每个提取数据点的可信度。

使用输出概率: 对于NER模型，通常可以直接利用模型输出的概率值作为置信度评分。即对于每个标签，选择最大概率作为置信度。

设定阈值: 设定一个置信度阈值，例如70%。只有置信度高于该阈值的数据才被认为是有效的提取结果。

动态调整: 随着模型的训练和数据的增加，可以动态调整置信度评分的计算方法和阈值。

以下是一个使用NER模型输出概率计算置信度的示例。

In [ ]:
import numpy as np

def calculate_confidence(predictions):
    """
    计算置信度评分

    Args:
        predictions (list): 模型输出的概率列表

    Returns:
        float: 置信度评分
    """
    max_prob = np.max(predictions)
    return max_prob

# 示例NER模型输出的概率
predictions = [0.1, 0.3, 0.6]  # 代表标签的概率
confidence_score = calculate_confidence(predictions)
print("置信度评分:", confidence_score)

# 判断是否通过置信度阈值
threshold = 0.7
if confidence_score > threshold:
    print("提取结果可信")
else:
    print("提取结果不可信，需进一步审核")

##### 3. 计算NER模型的精确率、召回率和F1分数

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 示例真实标签和预测标签
true_labels = ["B-PER", "O", "B-LOC", "O", "B-PER", "I-PER"]
predicted_labels = ["B-PER", "O", "O", "B-LOC", "B-PER", "I-PER"]

# 计算指标
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"精确率: {precision:.2f}")
print(f"召回率: {recall:.2f}")
print(f"F1分数: {f1:.2f}")

##### 4. 交叉验证
步骤：

准备数据集: 确保数据集包含已标注的文本数据，通常以BIO格式进行标注。

划分数据集: 将数据集划分为K个子集，K的值通常为5或10。每次迭代时，选择一个子集作为验证集，其余子集作为训练集。

训练和评估模型: 在每次迭代中，使用训练集训练模型，然后在验证集上评估模型性能，记录性能指标（如精确率、召回率和F1分数）。

汇总结果: 在所有迭代完成后，计算平均性能指标，以得到模型的整体表现。

以下是一个使用Python和scikit-learn实现K折交叉验证的代码：

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import BertForTokenClassification, BertTokenizer, Trainer, TrainingArguments

# 假设有一个已标注的NER数据集
# 示例数据集
data = {
    'text': ["Hawking was born in Oxford.", "Einstein was born in Ulm."],
    'bio_labels': [["B-PER", "O", "O", "O", "B-LOC", "O"], ["B-PER", "O", "O", "O", "B-LOC", "O"]]
}
df = pd.DataFrame(data)

# 初始化模型和tokenizer
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 计算K折交叉验证
k = 2
kf = KFold(n_splits=k)

# 存储所有折的评估结果
all_precision, all_recall, all_f1 = [], [], []

for train_index, test_index in kf.split(df):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]
    
    # 在训练集上训练模型（需要实现训练过程，略去）
    # 这里可以设置训练参数，训练模型
    # trainer.train()

    # 在测试集上评估模型
    # 假设此处预测的 labels 和 true_labels 生成
    # 这里进行预测，获取 predicted_labels 和 true_labels
    predicted_labels = []  # 通过模型得到的预测标签
    true_labels = []      # 测试集的真实标签

    # 计算指标
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    all_precision.append(precision)
    all_recall.append(recall)
    all_f1.append(f1)

# 输出平均评估结果
print(f"平均精确率: {np.mean(all_precision):.2f}")
print(f"平均召回率: {np.mean(all_recall):.2f}")
print(f"平均F1分数: {np.mean(all_f1):.2f}")


##### 5. 对照ESG标准框架
对照已知的ESG框架和标准（如SGX或GRI标准）进行验证。每个提取的实体（如某公司的温室气体排放数据）需要与标准中的相关类别进行匹配，确保其符合标准定义。

使用正则表达式或已标注的术语库比对提取信息和标准指标。

In [ ]:
# 读取SGX标准框架
esg_standards <- read.csv('sgx_esg_standards.csv')

# 对照提取数据的类别
validate_esg <- function(extracted_data, esg_standards) {
  matches <- list()
  for (data_point in extracted_data) {
    if (data_point$category %in% esg_standards$category) {
      matches <- append(matches, data_point)
    }
  }
  return(matches)
}

##### 6. 信心评分机制：
每次从报告中提取的信息都赋予一个“信心评分”，表示该信息的可靠性。评分可以基于提取模型的置信度、数据匹配的频率、以及与标准的符合程度。

In [ ]:
# 生成信心评分
generate_confidence_score <- function(data_point, model_confidence, match_with_standard) {
  if (match_with_standard) {
    score <- model_confidence * 1.2  # 匹配标准则增加20%的信心
  } else {
    score <- model_confidence
  }
  return(score)
}

##### 7. 人机交互

In [ ]:
library(shiny)

# 创建一个简单的审阅界面
ui <- fluidPage(
  titlePanel("ESG Data Review"),
  tableOutput("esg_table"),
  actionButton("approve", "Approve"),
  actionButton("reject", "Reject")
)

server <- function(input, output) {
  # 显示提取的数据
  output$esg_table <- renderTable({
    extracted_data
  })

  observeEvent(input$approve, {
    # 记录审核通过的结果
    approved_data <- append(approved_data, extracted_data)
  })

  observeEvent(input$reject, {
    # 记录审核不通过的结果
    rejected_data <- append(rejected_data, extracted_data)
  })
}

shinyApp(ui = ui, server = server)

##### 8. 适应不断变化的ESG标准与公司报告风格--迁移学习
使用预训练的语言模型（如BERT、GPT）进行迁移学习，适应新行业或新的ESG框架。可以基于新数据进行微调模型。

持续学习系统：
实现一个持续学习的管道，每当有新数据输入，模型会自动更新和优化。
设计自动反馈机制，从人类审核中提取反馈，进一步优化模型性能。

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

# 加载预训练的BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)

# 准备新的训练数据（假设已经有一个新的数据集）
new_texts = [...]  # 新的ESG文本
new_labels = [...]  # 适当的标签

# 创建数据集
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, truncation=True, padding='max_length', return_tensors='pt')
        encoding['labels'] = torch.tensor(label)
        return {key: val.squeeze() for key, val in encoding.items()}

new_dataset = CustomDataset(new_texts, new_labels)

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    num_train_epochs=3,
)

# 使用Trainer进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_dataset,
)

# 开始微调
trainer.train()


# 二、示例文本

In [ ]:
# 示例文本
esg_report_text = """
In 2023, our company achieved significant milestones in sustainability. We reduced our greenhouse gas emissions by 25% compared to the previous year. Additionally, 50% of our energy consumption now comes from renewable sources, demonstrating our commitment to clean energy. In terms of water usage, we managed to decrease water consumption by 10% in our operations.
"""

# 模拟的ESG提取信息
extracted_data = [
    {"category": "GHG Emissions", "value": "25% reduction", "unit": "%"},
    {"category": "Renewable Energy", "value": "50% consumption", "unit": "%"},
    {"category": "Water Consumption", "value": "10% reduction", "unit": "%"}
]


# 定义验证标准 (GRI framework or any other standards)
esg_standards = {
    "GHG Emissions": "Reduction of greenhouse gas emissions",
    "Renewable Energy": "Increase in renewable energy usage",
    "Water Consumption": "Reduction in water consumption"
}

# 计算置信度
def validate_extracted_data(extracted_data, esg_standards):
    validated_results = []
    for data in extracted_data:
        category = data["category"]
        value = data["value"]
        
        # 初始置信度设为1.0
        confidence_score = 1.0
        
        # 如果该类别在标准中有定义，增加置信度
        if category in esg_standards:
            confidence_score += 0.2  # 符合标准，增加置信度
        
        validated_results.append({
            "category": category,
            "value": value,
            "standard": esg_standards.get(category, "No standard found"),
            "confidence": confidence_score
        })
    
    return validated_results

# 调用验证函数
validated_data = validate_extracted_data(extracted_data, esg_standards)

# 显示验证结果
for item in validated_data:
    print(f"Category: {item['category']}, Value: {item['value']}, Standard: {item['standard']}, Confidence: {item['confidence']}")


In [ ]:
esg_report_text = """
In 2023, our company reduced carbon emissions by 20%, increased renewable energy usage to 60%, and improved waste recycling efficiency by 15%.
"""
# 模拟模型提取的 ESG 信息
extracted_data = [
    {"category": "Carbon Emissions", "value": "20% reduction", "confidence": 0.9},
    {"category": "Renewable Energy", "value": "60% increase", "confidence": 0.85},
    {"category": "Waste Recycling", "value": "15% efficiency", "confidence": 0.8}
]

# 模拟专家提供的反馈
def human_in_the_loop_feedback(extracted_data):
    print("Initial Model Output:")
    for data in extracted_data:
        print(f"Category: {data['category']}, Value: {data['value']}, Confidence: {data['confidence']}")
    
    # 模拟专家的修改
    print("\nExpert Feedback:")
    feedback_data = [
        {"category": "Carbon Emissions", "value": "25% reduction", "confidence": 0.95},
        {"category": "Renewable Energy", "value": "65% increase", "confidence": 0.90},
        {"category": "Waste Recycling", "value": "20% efficiency", "confidence": 0.85}
    ]
    
    # 返回修正后的数据
    return feedback_data

# 专家反馈并更新模型结果
updated_data = human_in_the_loop_feedback(extracted_data)

# 输出修正后的数据
print("\nUpdated Model Output after Expert Feedback:")
for data in updated_data:
    print(f"Category: {data['category']}, Value: {data['value']}, Confidence: {data['confidence']}")


In [ ]:
# BERT-序列分类
esg_report_text = """
Our company aims to achieve carbon neutrality by 2030, and in 2023, we successfully increased our renewable energy usage to 75%.
"""
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch

# 加载预训练的 BERT 模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3类ESG类别

# 模拟 ESG 报告的分类任务
esg_reports = [
    "Our company aims to reduce carbon emissions by 25% by 2025.",
    "In 2023, we increased renewable energy usage to 75%.",
    "Water usage was reduced by 10%."
]

# Tokenize 输入文本
inputs = tokenizer(esg_reports, return_tensors="pt", padding=True, truncation=True)

# 模拟分类标签 (0: Carbon Emissions, 1: Renewable Energy, 2: Water Usage)
labels = torch.tensor([0, 1, 2])

# 定义优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 微调模型（1个 epoch）
model.train()
outputs = model(**inputs, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

# 模拟新的 ESG 报告来测试微调后的模型
new_report = "The company aims to achieve carbon neutrality by 2030."
inputs = tokenizer(new_report, return_tensors="pt", padding=True, truncation=True)
model.eval()
with torch.no_grad():
    predictions = model(**inputs).logits
    predicted_class = torch.argmax(predictions, dim=1).item()

# 输出预测的类别
categories = ["Carbon Emissions", "Renewable Energy", "Water Usage"]
print(f"Predicted Category: {categories[predicted_class]}")
